In [41]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from pathlib import Path

In [111]:
# Read data csv file
file = Path('/Users/CollinHeist/Downloads/Media - Data CSVs.csv/')

# Parse as DF object
df = pd.read_csv(
    file,
    header=None,                                       # No header row 
    names=['Date', 'Start Time', 'End Time', 'Title'], # Column titles
    na_values='-',                                     # Value "-" is converted to NA
    usecols=list(range(4)),                            # Use first four columns
)

# Forward-fill the dates
df['Date'] = pd.to_datetime(df['Date']).ffill(axis=0)

# Convert time to datetime objects with times + date
def row_to_datetime(row: dict, time_key: str) -> datetime:
    """
    Convert a row of a DF to a single combined Datetime object. This adds
    the row's 'Date' value to row's value from `time_key`. If this operation
    fails, 0.0 is returned instead.
    """
    
    try:
        return row['Date'] + timedelta(minutes=float(row[time_key]) * 60 * 24)
    except:
        return row['Date']

df['Start Time'] = df.apply(lambda row: row_to_datetime(row, 'Start Time'), axis=1)
df['End Time'] = df.apply(lambda row: row_to_datetime(row, 'End Time'), axis=1)

In [112]:
np.sum(df['End Time'] - df['Start Time'])

Timedelta('135 days 10:54:52.992000')

In [115]:
df[df['Title'] == 'The Office']

,Date,Start Time,End Time,Title
11,2015-10-28,2015-10-28 15:15:59.904,2015-10-28 15:58:00.192,The Office
12,2015-10-28,2015-10-28 15:58:00.192,2015-10-28 16:39:59.616,The Office
13,2015-10-28,2015-10-28 16:41:59.712,2015-10-28 17:24:00.000,The Office
20,2015-10-30,2015-10-30 17:55:00.192,2015-10-30 18:36:59.616,The Office
21,2015-10-30,2015-10-30 18:36:59.616,2015-10-30 19:18:59.904,The Office
24,2015-10-30,2015-10-30 20:48:00.288,2015-10-30 21:09:00.000,The Office
25,2015-10-30,2015-10-30 21:09:00.000,2015-10-30 21:31:00.192,The Office
26,2015-10-30,2015-10-30 21:31:00.192,2015-10-30 21:53:00.384,The Office
34,2015-11-02,2015-11-02 21:49:00.192,2015-11-02 22:11:00.384,The Office
35,2015-11-02,2015-11-02 22:21:00.000,2015-11-02 22:43:00.192,The Office
